In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [18]:
dataset = '~/Documents/datascience/data/bq-results-20250811-183005-1754937037763.csv'
df = pd.read_csv(dataset)

In [19]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880299 entries, 0 to 880298
Data columns (total 48 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   numericid                        880299 non-null  int64  
 1   transactiontag                   880299 non-null  object 
 2   transactionId                    880299 non-null  int64  
 3   txn_create_ts                    880299 non-null  object 
 4   transaction_date                 880299 non-null  object 
 5   transaction_month                880299 non-null  object 
 6   txn_amount                       880299 non-null  float64
 7   hasACHreturn                     677440 non-null  object 
 8   return_ach_txn_create_ts         74416 non-null   object 
 9   return_date                      74416 non-null   object 
 10  return_month                     74416 non-null   object 
 11  ach_returnCode                   74404 non-null   object 
 12  ac

In [20]:
df['tranaction_time'] = pd.to_datetime(df['txn_create_ts'], errors='coerce')
df['return_time'] = pd.to_datetime(df['return_ach_txn_create_ts'], errors='coerce')
df['is_fraud_return'] = df['hasACHreturn'] == True
df['return_days_lag'] = (df['return_time'] -  df['tranaction_time']).dt.days

In [22]:
df.head(100)

,numericid,transactiontag,transactionId,txn_create_ts,transaction_date,transaction_month,txn_amount,hasACHreturn,return_ach_txn_create_ts,return_date,...,count_deposits_24hours,count_deposits_48hours,plaid_availableBalance,plaid_currentBalance,holdtime,decision,tranaction_time,return_time,is_fraud_return,return_days_lag
0,4180046198930931713,d3dfcedb-49c0-4d22-b67a-a05b049db63c,0,2024-08-01 15:25:11.789 UTC,2024-08-01,2024-08,500.0,NaN,NaN,NaN,...,1,7,0.00,-15600.78,3,Reject,2024-08-01 15:25:11.789000+00:00,NaT,False,NaN
1,1732979523948904449,c1ff9419-87dc-4af0-853d-706bb325e9df,12650711204,2024-08-01 15:25:02.782 UTC,2024-08-01,2024-08,1000.0,False,NaN,NaN,...,1,1,1587.79,4155.17,3,Allow,2024-08-01 15:25:02.782000+00:00,NaT,False,NaN
2,8651500006135562241,d187c4f8-29d2-43bf-a3e2-8040db58de54,12650708391,2024-08-01 15:24:47.375 UTC,2024-08-01,2024-08,500.0,False,NaN,NaN,...,1,1,NaN,NaN,3,Allow,2024-08-01 15:24:47.375000+00:00,NaT,False,NaN
3,3443200523002445825,f5986506-25f5-4521-92c7-4623c30591e6,12650705814,2024-08-01 15:24:30.515 UTC,2024-08-01,2024-08,72.9,False,NaN,NaN,...,1,1,NaN,NaN,3,Allow,2024-08-01 15:24:30.515000+00:00,NaT,False,NaN
4,3504079927781621761,1ba3c51d-9429-4ea7-b690-ab6643c63737,12650703416,2024-08-01 15:24:18.249 UTC,2024-08-01,2024-08,100.0,True,2024-08-02 19:47:01.917 UTC,2024-08-02,...,1,1,NaN,NaN,3,Allow,2024-08-01 15:24:18.249000+00:00,2024-08-02 19:47:01.917000+00:00,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,7642566107601043457,5c750907-2672-409b-800c-b8c36ef13ee0,12650366430,2024-08-01 14:48:59.504 UTC,2024-08-01,2024-08,1200.0,False,NaN,NaN,...,1,1,NaN,NaN,3,Allow,2024-08-01 14:48:59.504000+00:00,NaT,False,NaN
96,7894619441642602497,572870ea-cefd-4f39-990b-4f9e6f741931,12650361005,2024-08-01 14:47:56.924 UTC,2024-08-01,2024-08,20.0,False,NaN,NaN,...,1,1,6111.81,6293.92,3,Allow,2024-08-01 14:47:56.924000+00:00,NaT,False,NaN
97,9033075891533512705,66316e8f-6743-468e-bf1c-4305d0962aa5,12650357198,2024-08-01 14:47:13.938 UTC,2024-08-01,2024-08,200.0,False,NaN,NaN,...,1,1,NaN,NaN,3,Allow,2024-08-01 14:47:13.938000+00:00,NaT,False,NaN
98,3067215632756899841,4deb19ac-9b6d-4f7d-862a-d3570926554e,12650356575,2024-08-01 14:47:06.422 UTC,2024-08-01,2024-08,5.0,False,NaN,NaN,...,1,1,216.47,214.49,3,Allow,2024-08-01 14:47:06.422000+00:00,NaT,False,NaN


In [14]:
df['ach_returnCode'].value_counts(normalize=True)


ach_returnCode
R01    0.697960
R02    0.113112
R03    0.056018
R16    0.047202
R10    0.025241
R08    0.021894
R04    0.014623
R20    0.010591
R09    0.008776
R11    0.001962
R07    0.001962
R29    0.000269
R17    0.000175
R45    0.000054
R13    0.000040
R15    0.000040
R06    0.000040
R05    0.000027
R24    0.000013
Name: proportion, dtype: float64

In [25]:
total_txn = len(df)
total_vol = df['txn_amount'].sum()